In [525]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

    
downloaded = drive.CreateFile({'id': '1d2WaIX0s4zzZE8AzBIbvEHGXe0Bmlp_x'})
downloaded.GetContentFile('train.csv')
downloaded = drive.CreateFile({'id': '1mPhuUHZ80SB30Tvtdb-4q0q81ut9Pxhz'})
downloaded.GetContentFile('val.csv')
downloaded = drive.CreateFile({'id': '1y_j8bhtU1xg0SOOLDEopdrQOE0w_LPRp'})
downloaded.GetContentFile('test_without_labels.csv')

import numpy as np
import pandas as pd
df_train = pd.read_csv('train.csv')
df_train.sample(10)

,sents,labels
256,"ISIL has had control of Mosul for two years , ...",B-Organisation O O O O B-Location O B-Temporal...
512,The UK is at the forefront of global efforts t...,B-Organisation I-Organisation O O O O O O O O ...
557,The attacks occurred during daylight hours fro...,O O O O B-Temporal I-Temporal I-Temporal I-Tem...
111,The fortified city fell to coalition forces af...,B-Location I-Location I-Location O O B-Organis...
441,"Be in no doubt , we are committed to destroyin...",O O O O O B-Organisation O O O O O O O O B-Org...
414,It only underscores the importance of Coalitio...,O O O O O O B-Organisation O O O B-Organisatio...
274,"""",O
314,Our policies are designed to assist ordinary c...,B-Organisation O O O O O B-Organisation I-Orga...
233,The main purpose of the dam is to divert flood...,O O O O B-Location I-Location O O O O O B-Loca...
435,We had more workers and we started to cultivat...,B-Organisation O O O O B-Organisation O O O O O O


In [526]:
df_val = pd.read_csv('val.csv')
df_val.sample(10)

,sents,labels
139,""" The army was quoted as warning Israel "" "" of...",O B-Organisation I-Organisation O O O O B-Orga...
185,It violated the peace of Ramadan .,O O O O O B-Temporal O
66,The use of chemical weapons by Daesh in Marea ...,O O O B-Weapon I-Weapon O B-Organisation O B-L...
95,"Sadiq draws me aside , as Karim continues to p...",B-Person O B-Person O O O B-Person O O O O O O...
46,""" "" The camp where they live is oppressive but...",O O B-Location I-Location O B-Organisation O O...
37,Navy Senior Chief Petty Officer Scott C . Dayt...,B-Person I-Person I-Person I-Person I-Person I...
164,Syrians at the farm come from a nearby refugee...,B-Organisation I-Organisation I-Organisation I...
168,He told the tribunal that he was instead kidna...,B-Person O B-Organisation I-Organisation O B-P...
126,"The group plans to be deployed two months , wh...",B-Organisation I-Organisation O O O O B-Tempor...
167,Minister Ellwood said :,B-Person I-Person O O


In [527]:
df_test = pd.read_csv('test_without_labels.csv')
df_test.sample(10)

,sents
181,All aircraft returned to base safely .
72,"In Syria , the press secretary said , Syrian d..."
118,ISIL was using the hospital as a base of opera...
75,These atrocities show once again the utter dis...
111,The airstrikes have destroyed 967 vehicle bomb...
97,We express our deep condolences to family and ...
78,The Foreign Secretary Boris Johnson joined US ...
119,The Hamrin Mountains ( Arabic : ج ب ل ح م ر ي ...
121,""" "" "" I do planting , weeding and harvesting o..."
113,He said the groundwork was being laid for the ...


In [528]:
sent_train = df_train['sents'].tolist()
label_train = df_train['labels'].tolist()
sent_val = df_val['sents'].tolist()
label_val = df_val['labels'].tolist()
sent_test = df_test['sents'].tolist()
print(sent_train)

['Operation Steel Curtain ( Arabic : ا ل ح ج ا ب ا ل ف و ل ا ذ ي Al Hejab Elfulathi ) was a military operation executed by coalition forces in early November 2005 to reduce the flow of foreign insurgents crossing the border and joining the Iraqi insurgency .', 'The hospital has facilities for MRI and CT scan , a gamma camera and lithotripsy device .', 'The operation was important in that it was the first large scale deployment of the New Iraqi Army .', 'This was my first visit to Uzbekistan and an important opportunity to deepen and strengthen bilateral relations between the UK and Uzbekistan on a number of issues important to us both .', 'The group was founded by Sheikh Abu Omar al - Ansari . "', 'And hopefully , by reading , we can achieve this . " " "', 'But nobody can leave now .', 'The Members of the Security Council are encouraged by the decision of Iraqi President Fuad Masum to nominate a new Prime Minister - designate .', 'Iraq', 'We extend our deepest condolences to the famili

In [529]:
sent_train = [s.lower() for s in sent_train]

sent_val = [s.lower() for s in sent_val]

sent_test = [s.lower() for s in sent_test]

In [530]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

from nltk.tokenize import word_tokenize, sent_tokenize

def token_sent(datalist):
  lemmatizer = WordNetLemmatizer()
  tokenized_sentences = []
  for sentence in datalist:
    tokens = sentence.split(" ")
    tokenized_sentences.append(tokens)
  text_train_le=[]
  for tokens in tokenized_sentences:
    lemma_sentence = [lemmatizer.lemmatize(w) for w in tokens ]
    text_train_le.append(lemma_sentence)
  return text_train_le


def token_label(datalist):
  tokenized_sentences = []
  for sentence in datalist:
    tokens = sentence.split(" ")
    tokenized_sentences.append(tokens)
  return tokenized_sentences

train_sent = token_sent(sent_train)
train_label = token_label(label_train)
val_sent = token_sent(sent_val)
val_label = token_label(label_val)
test_sent = token_sent(sent_test)
print(train_sent)
print(train_label)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[['operation', 'steel', 'curtain', '(', 'arabic', ':', 'ا', 'ل', 'ح', 'ج', 'ا', 'ب', 'ا', 'ل', 'ف', 'و', 'ل', 'ا', 'ذ', 'ي', 'al', 'hejab', 'elfulathi', ')', 'wa', 'a', 'military', 'operation', 'executed', 'by', 'coalition', 'force', 'in', 'early', 'november', '2005', 'to', 'reduce', 'the', 'flow', 'of', 'foreign', 'insurgent', 'crossing', 'the', 'border', 'and', 'joining', 'the', 'iraqi', 'insurgency', '.'], ['the', 'hospital', 'ha', 'facility', 'for', 'mri', 'and', 'ct', 'scan', ',', 'a', 'gamma', 'camera', 'and', 'lithotripsy', 'device', '.'], ['the', 'operation', 'wa', 'important', 'in', 'that', 'it', 'wa', 'the', 'first', 'large', 'scale', 'deployment', 'of', 'the', 'new', 'iraqi', 'army', '.'], ['this', 'wa', 'my', 'first', 'visit', 'to', 'uzbekistan', 'and'

In [531]:
# This part of code is modified from lab9
word_to_ix = {}
for sentence in train_sent+val_sent+test_sent:
    for word in sentence:
        word = word.lower()
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
word_list = list(word_to_ix.keys())

START_TAG = "<START>"
STOP_TAG = "<STOP>"
tag_to_ix = {START_TAG:0, STOP_TAG:1}
for tags in train_label:
    for tag in tags:
        if tag not in tag_to_ix:
            tag_to_ix[tag] = len(tag_to_ix)

In [532]:
# This part of code is modified from lab7
import spacy
nlp = spacy.load("en_core_web_sm")

def get_dependence_parse(sentence):
  dependence_parse=[]
  for sent in sentence:
    doc = nlp(' '.join(sent))
    dependence_temp = []
    for token in doc:
      token_dep = token.dep_
      dependence_temp.append(token_dep)
    dependence_parse.append(dependence_temp)

  return dependence_parse


In [533]:
dep_sentence = get_dependence_parse(train_sent+val_sent+test_sent)

In [534]:
sentence_wv = dep_sentence
size_wv = 50
window_wv = 5
min_count_wv = 1
workers_wv = 2
sg_wv = 0
wv_dep_model = Word2Vec(sentences=sentence_wv, size=size_wv, window=window_wv, min_count=min_count_wv, workers=workers_wv, sg=sg_wv)

In [535]:
# This part of code is modified from lab1
from collections import Counter
import math

def get_TF_IDF(sentence):
  DF = {}
  for sent in sentence:
    for token in np.unique(sent):
      try:
            DF[token] +=1
      except:
            DF[token] =1

  tf_idf = {}

  N = len(sentence) 

  for sent in sentence:
      counter = Counter(sent)
      total_num_words = len(sent)    
      for term in np.unique(sent):
          tf = counter[term]/total_num_words
          df = DF[term]
          idf = math.log(N/(df+1))+1
          tf_idf[term] = tf*idf
  return tf_idf


In [536]:
TFIDF_sentence = get_TF_IDF(train_sent+val_sent+test_sent)

In [537]:
word_dep={}
for i in range(len(sentences)):
  for j in range(len(sentences[i])):
    word_dep[sentences[i][j]]= dep_sentence[i][j]
print(word_dep)


{'operation': 'dobj', 'steel': 'compound', 'curtain': 'appos', '(': 'advmod', 'arabic': 'parataxis', ':': 'dobj', 'ا': 'ROOT', 'ل': 'ROOT', 'ح': 'ROOT', 'ج': 'punct', 'ب': 'ROOT', 'ف': 'punct', 'و': 'compound', 'ذ': 'compound', 'ي': 'ROOT', 'al': 'compound', 'hejab': 'compound', 'elfulathi': 'appos', ')': 'dep', 'wa': 'nsubj', 'a': 'det', 'military': 'amod', 'executed': 'acl', 'by': 'agent', 'coalition': 'compound', 'force': 'compound', 'in': 'prep', 'early': 'advmod', 'november': 'pobj', '2005': 'nummod', 'to': 'aux', 'reduce': 'advcl', 'the': 'det', 'flow': 'dobj', 'of': 'prep', 'foreign': 'amod', 'insurgent': 'amod', 'crossing': 'acl', 'border': 'pobj', 'and': 'cc', 'joining': 'ROOT', 'iraqi': 'amod', 'insurgency': 'dobj', '.': 'punct', 'hospital': 'nsubj', 'ha': 'punct', 'facility': 'dobj', 'for': 'prep', 'mri': 'pobj', 'ct': 'compound', 'scan': 'conj', ',': 'punct', 'gamma': 'nmod', 'camera': 'nmod', 'lithotripsy': 'compound', 'device': 'amod', 'important': 'amod', 'that': 'mark',

In [538]:
import gensim.downloader as api
word_emb_model = api.load("glove-twitter-100") 

In [539]:
#Please change input_embedding to 
#                                   Semantic -- for only semantic input
#                                   Semantic + DP -- for semantic and dependency parsing input
#                                   Semantic + TFIDF -- for semantic and TFIDF input
input_embedding = "Semantic all"

In [540]:
# This part of code is modified from lab9

EMBEDDING_DIM = 151

embedding_matrix = []
for word in word_list:
    try:
        if input_embedding == "Semantic":
          EMBEDDING_DIM = 100
          word_vector = list(word_emb_model.wv[word])

        elif input_embedding == "Semantic + DP":
          EMBEDDING_DIM = 150
          word_vector = list(word_emb_model.wv[word])+ list(wv_dep_model.wv[word_dep[word]])
        elif input_embedding == "Semantic + TFIDF":
          EMBEDDING_DIM = 101
          word_vector = list(word_emb_model.wv[word])
          word_vector.append(TFIDF_sentence[word])
        else:
          word_vector = list(word_emb_model.wv[word])+ list(wv_dep_model.wv[word_dep[word]])
          word_vector.append(TFIDF_sentence[word])
        embedding_matrix.append(word_vector)

    except:
        embedding_matrix.append([0]*EMBEDDING_DIM)
embedding_matrix = np.array(embedding_matrix)
embedding_matrix.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


(3641, 151)

In [541]:
len(word_to_ix)

3641

In [542]:
# This part of code is modified from lab9
def to_index(data, to_ix):
    input_index_list = []
    for sent in data:
        input_index_list.append([to_ix[w] for w in sent])
    return input_index_list

train_input_index =  to_index(train_sent,word_to_ix)
train_output_index = to_index(train_label,tag_to_ix)
val_input_index = to_index(val_sent,word_to_ix)
val_output_index = to_index(val_label,tag_to_ix)
test_input_index = to_index(test_sent,word_to_ix)
input_data = train_input_index+val_input_index
output_data = train_output_index + val_output_index

In [545]:
# This part of code is modified from lab9 and lab10
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

torch.manual_seed(1)

def argmax(vec):

    _, idx = torch.max(vec, 1)
    return idx.item()


def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim, num_layer, attention_method, crf, dropout=0.2):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.dropout = dropout
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.num_layer = num_layer
        self.attention_method = attention_method
        self.crf = crf

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        self.dropout = nn.Dropout(self.dropout)
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=num_layer, bidirectional=True)
        if attention_method == "DOT_PRODUCT" or attention_method == "SCALE_DOT_PRODUCT" or attention_method == "COSINE":
            self.hidden2tag = nn.Linear(hidden_dim*2, self.tagset_size)
        else:
            self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2*self.num_layer, 1, self.hidden_dim // 2).to(device),
                torch.randn(2*self.num_layer, 1, self.hidden_dim // 2).to(device))
      


    def _forward_alg(self, feats):

        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)

        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        forward_var = init_alphas

        for feat in feats:
            alphas_t = []  
            for next_tag in range(self.tagset_size):
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                trans_score = self.transitions[next_tag].view(1, -1)
                next_tag_var = forward_var + trans_score + emit_score
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha


    def cal_attention(self, lstm_out, attention_method):
      hidden_2 = lstm_out.permute(1, 2, 0)
      hidden_1 = lstm_out.permute(1, 0, 2)
      if attention_method == "DOT_PRODUCT":
          attn_weights = F.softmax(torch.bmm(hidden_1, hidden_2),dim=-1)
          hidden_2 = lstm_out.permute(1, 0, 2)
          attn_output = torch.bmm(attn_weights, hidden_2)
          concat_output = torch.cat((attn_output[0], hidden_1[0]), 1)

      elif attention_method == "SCALE_DOT_PRODUCT":
          attn_weights = F.softmax(1/np.sqrt(self.hidden_dim)*torch.bmm(hidden_1, hidden_2),dim=-1)
          hidden_2 = lstm_out.permute(1, 0, 2)
          attn_output = torch.bmm(attn_weights, hidden_2)
          concat_output = torch.cat((attn_output[0], hidden_1[0]), 1)
      elif attention_method == "COSINE":
          hidden_1_tem = hidden_1.permute(1,2,0)
          cos = nn.CosineSimilarity(dim=1, eps=1e-6)
          attn_weights = cos(hidden_1_tem,hidden_2)
          attn_weights = attn_weights.view(1, attn_weights.size(0), attn_weights.size(0))
          hidden_2 = lstm_out.permute(1, 0, 2)
          attn_output = torch.bmm(attn_weights, hidden_2)
          concat_output = torch.cat((attn_output[0], hidden_1[0]), 1)
      else:
          concat_output = lstm_out
      return concat_output


    def _get_lstm_features(self, sentence):
        
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        embeds = self.dropout(embeds)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        if self.attention_method == "DOT_PRODUCT" or self.attention_method == "SCALE_DOT_PRODUCT" or self.attention_method == "COSINE":
            att_out = self.cal_attention(lstm_out, self.attention_method)
            lstm_feats = F.log_softmax(self.hidden2tag(att_out), dim=1)
        else:
          lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
          lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats



    def _score_sentence(self, feats, tags):
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        forward_var = init_vvars
        for feat in feats:
            bptrs_t = [] 
            viterbivars_t = []  

            for next_tag in range(self.tagset_size):
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))

            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)


        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG] 
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  
        lstm_feats = self._get_lstm_features(sentence)

        if self.crf == True:
          score, tag_seq = self._viterbi_decode(lstm_feats)
          return score, tag_seq
        else:
          score, tag = torch.max(lstm_feats, dim = -1) 
          return score, tag



In [546]:
# This part of code is modified from lab9
def cal_acc(model, input_index, output_index):
    ground_truth = []
    predicted = []
    for i,idxs in enumerate(input_index):
        ground_truth += output_index[i]
        score, pred = model(torch.tensor(idxs, dtype=torch.long).to(device))
        predicted += pred
    accuracy = sum(np.array(ground_truth) == np.array(predicted))/len(ground_truth)
    return predicted, ground_truth, accuracy

In [547]:
# This part of code is modified from lab9
# In attention method, please change "SCALE_DOT_PRODUCT" to "DOT_PRODUCT" for dot product attention, to "COSINE" for cosine similarity attention.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 200

model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, 2, "SCALE_DOT_PRODUCT", True, dropout = 0.2).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

In [548]:
# This part of code is modified from lab9
"""Each epoch will take about 1-2 minutes"""

import datetime

for epoch in range(20):  
    time1 = datetime.datetime.now()
    train_loss = 0


    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]
        model.zero_grad()
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))


Epoch:1, Training loss: 15775.15, train acc: 0.7188, val loss: 5071.52, val acc: 0.6670, time: 122.11s
Epoch:2, Training loss: 10649.92, train acc: 0.7746, val loss: 3895.02, val acc: 0.7294, time: 121.03s
Epoch:3, Training loss: 8068.72, train acc: 0.8084, val loss: 3187.46, val acc: 0.7582, time: 120.64s
Epoch:4, Training loss: 6557.02, train acc: 0.8251, val loss: 2752.28, val acc: 0.7704, time: 119.90s
Epoch:5, Training loss: 5524.34, train acc: 0.8443, val loss: 2422.52, val acc: 0.7871, time: 120.47s
Epoch:6, Training loss: 4918.44, train acc: 0.8505, val loss: 2260.65, val acc: 0.7969, time: 120.11s
Epoch:7, Training loss: 4347.20, train acc: 0.8680, val loss: 2100.38, val acc: 0.8038, time: 120.48s
Epoch:8, Training loss: 3978.31, train acc: 0.8834, val loss: 2030.43, val acc: 0.8093, time: 120.93s
Epoch:9, Training loss: 3498.39, train acc: 0.8895, val loss: 2052.07, val acc: 0.8030, time: 121.17s
Epoch:10, Training loss: 3252.48, train acc: 0.9024, val loss: 1963.91, val acc:

In [553]:
torch.save(model, 'bestmodel.pt')

In [554]:
# This part of code is modified from lab9
y_pred, y_true, _ = cal_acc(model,val_input_index,val_output_index)

def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]

y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)

In [555]:
# This part of code is modified from lab9
from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))

                     precision    recall  f1-score   support

B-DocumentReference     0.5000    0.2500    0.3333        20
         B-Location     0.6442    0.7204    0.6802       186
 B-MilitaryPlatform     0.0000    0.0000    0.0000        16
            B-Money     0.5000    0.2000    0.2857         5
      B-Nationality     0.3333    0.1250    0.1818         8
     B-Organisation     0.7122    0.7071    0.7097       280
           B-Person     0.7768    0.8529    0.8131       102
         B-Quantity     0.6667    0.6909    0.6786        55
         B-Temporal     0.7727    0.7234    0.7473        47
           B-Weapon     0.3226    0.2632    0.2899        38
I-DocumentReference     0.8235    0.3373    0.4786        83
         I-Location     0.6391    0.6415    0.6403       265
 I-MilitaryPlatform     0.0000    0.0000    0.0000        16
            I-Money     1.0000    0.4000    0.5714        10
      I-Nationality     0.0000    0.0000    0.0000         1
     I-Organisation    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [551]:
test_result_length = len(y_pred_decode)
print(test_result_length)

5274


In [552]:
test_result = {'Id':range(test_result_length) ,'predicted': y_pred_decode }
df = pd.DataFrame(test_result)
df.to_csv('./test_result.csv', index=False)